## Casualty Analysis of Speeding Cases resulting in People Killed outside of Vehicle

There are 36,560 people died in car accidents in 2018, and 26% of the cases are speeding related.

### Research Question
In this notebook we will be analyzing speeding-related traffic crashes in 2015 that resulting the deaths outside of vehicle and exploring the contribution factors. We will use machine learning algorithms to analyze the data.

### Data Loading

The dataset is from Google BigQuery public data [NHTSA Traffic Fatalities](https://console.cloud.google.com/marketplace/product/nhtsa-data/nhtsa-traffic-fatalities?filter=solution-type:dataset&filter=category:public-safety&id=412c019d-62ba-4ee9-af00-04183e9ddb84) and [How to connect to Google BigQuery](https://cloud.google.com/bigquery/docs/quickstarts?hl=en_US)
For more detail about the dataset please visit [NHTSA Fatality Analysis Reporting System](https://www.nhtsa.gov/research-data/fatality-analysis-reporting-system-fars)
Here is the sample SQL code I used to query the data from BigQuery:
```
    SELECT
    ap.*, body_type, vehicle_trailing, gross_vehicle_weight_rating, travel_speed, previous_recorded_crashes,
    previous_recorded_suspensions_and_revocations, previous_dwi_convictions, previous_speeding_convictions,
    related_factors_driver_level_1, related_factors_driver_level_2, related_factors_driver_level_3,
    related_factors_driver_level_4, roadway_alignment, roadway_grade, roadway_surface_type, roadway_surface_condition,
    crash_type, fatalities_in_vehicle
    FROM (
    SELECT a.state_number, a.consecutive_number,number_of_forms_submitted_for_persons_not_in_motor_vehicles,
    a.county, city, a.day_of_crash, a.month_of_crash, day_of_week, a.hour_of_crash, national_highway_system,
    a.land_use, a.functional_system, route_signing, a.first_harmful_event, a.manner_of_collision,
    relation_to_junction_within_interchange_area, relation_to_junction_specific_location, type_of_intersection,
    relation_to_trafficway, light_condition, atmospheric_conditions, related_factors_crash_level_1,
    related_factors_crash_level_2, related_factors_crash_level_3, number_of_fatalities, p.vehicle_number,
    p.person_number, p.rollover, p.age, p.sex, p.person_type, p.injury_severity, p.police_reported_alcohol_involvement,
    p.police_reported_drug_involvement, related_factors_person_level1, related_factors_person_level2,
    related_factors_person_level3, race
    FROM
    `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015` a
    LEFT JOIN
    `bigquery-public-data.nhtsa_traffic_fatalities.person_2015` p
    ON a.consecutive_number = p.consecutive_number
    WHERE p.person_type = 1
    ) ap
    JOIN
    `bigquery-public-data.nhtsa_traffic_fatalities.vehicle_2015` v
    ON ap.vehicle_number = v.vehicle_number
    WHERE speeding_related LIKE '%Yes%'
```
I wrote is simple [script](bq.py) to convert the data from Google Big Query to Data Frame